## Using Arrays to Define a Model

In  previous tutorial, we learned how to solve a MIP with just a few variables and constraints, which are defined individually. For larger problems, it's more convenient to define the variables and constraints by looping over arrays. The next example illustrates this.

__Note:__ The example is adopted from OR-Tools website.

### Example
In this example we'll solve the following problem.
<br>
<dl>
    <dt>Maximize 7<em>x<sub>1</sub></em> + 8<em>x<sub>2</sub></em> + 2<em>x<sub>3</sub></em> + 9<em>x<sub>4</sub></em> + 6<em>x<sub>5</sub></em>
        subject to the following constraints:</dt><br><br>
      <dd>
        <div class="devsite-table-wrapper"><table border="1" style="width:500px">
          <tbody><tr>
            <td>5<em>x<sub>1</sub></em></td>
            <td>+</td>
            <td>7<em>x<sub>2</sub></em></td>
            <td>+</td>
            <td>9<em>x<sub>3</sub></em></td>
            <td>+</td>
            <td>2<em>x<sub>4</sub></em></td>
            <td>+</td>
            <td>1<em>x<sub>5</sub></em></td>
            <td>≤</td>
            <td>250</td>
          </tr>
          <tr>
            <td>18<em>x<sub>1</sub></em></td>
            <td>+</td>
            <td>4<em>x<sub>2</sub></em></td>
            <td>-</td>
            <td>9<em>x<sub>3</sub></em></td>
            <td>+</td>
            <td>10<em>x<sub>4</sub></em></td>
            <td>+</td>
            <td>12<em>x<sub>5</sub></em></td>
            <td>≤</td>
            <td>285</td>
          </tr>
          <tr>
            <td>4<em>x<sub>1</sub></em></td>
            <td>+</td>
            <td>7<em>x<sub>2</sub></em></td>
            <td>+</td>
            <td>3<em>x<sub>3</sub></em></td>
            <td>+</td>
            <td>8<em>x<sub>4</sub></em></td>
            <td>+</td>
            <td>5<em>x<sub>5</sub></em></td>
            <td>≤</td>
            <td>211</td>
          </tr>
          <tr>
            <td>5<em>x<sub>1</sub></em></td>
            <td>+</td>
            <td>13<em>x<sub>2</sub></em></td>
            <td>+</td>
            <td>16<em>x<sub>3</sub></em></td>
            <td>+</td>
            <td>3<em>x<sub>4</sub></em></td>
            <td>-</td>
            <td>7<em>x<sub>5</sub></em></td>
            <td>≤</td>
            <td>315</td>
          </tr>
         </tbody></table></div>
      </dd>
    </dl>
  
<p>where <em>x<sub>1</sub></em>, <em>x<sub>2</sub></em>, ..., <em>x<sub>5</sub></em> are non-negative
integers.</p>

The following sections present programs that solve this problem. The programs use the same methods as the previous MIP example, but in this case apply them to array values in a loop.

### Install OR-Tools

In [ ]:
pip install --upgrade --user ortools

### Declare the solver
In any MIP program, you start by importing the linear solver wrapper and declaring the MIP solver, as shown in the previous MIP example.

In [3]:
from ortools.linear_solver import pywraplp

### Create the data
The following code creates arrays containing the data for the example: the variable coefficients for the constraints and objective function, and bounds for the constraints.

In [4]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data['bounds'] = [250, 285, 211, 315]
    data['obj_coeffs'] = [7, 8, 2, 9, 6]
    data['num_vars'] = 5
    data['num_constraints'] = 4
    return data


### Instantiate the data
The following code instantiates the data model.

In [5]:
data = create_data_model()

### Instantiate the solver
The following code instantiates the solver.

In [6]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

### Define the variables
The following code defines the variables for the example in a loop. For large problems, this is easier than defining the variables individually, as in the previous example.

In [7]:
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
print('Number of variables =', solver.NumVariables())

Number of variables = 5


### Define the constraints

The following code creates the constraints for the example, using loop.

This code assigns the coefficient of the variable
<code translate="no" dir="ltr">x[j]</code> in constraint <code translate="no" dir="ltr">i</code> to be the <code translate="no" dir="ltr">[i][j]</code> entry of the array
<code translate="no" dir="ltr">constraint_coeffs</code>.</p>

In [8]:
for i in range(data['num_constraints']):
    constraint_expr = []
    for j in range(data['num_vars']):
        constraint_expr.append(data['constraint_coeffs'][i][j] * x[j])
    solver.Add(sum(constraint_expr) <= data['bounds'][i])
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 4


### Define the objective
The following code defines the objective function for the example.

In [10]:
obj_expr = []
for j in range(data['num_vars']):
    obj_expr.append(data['obj_coeffs'][j] * x[j])
solver.Maximize(solver.Sum(obj_expr))

### Call the solver
The following code calls the solver.

In [11]:
status = solver.Solve()

### Display the solution
The following code displays the solution.

In [12]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        print(x[j].name(), ' = ', x[j].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 260.00000000000006
x[0]  =  9.0
x[1]  =  20.0
x[2]  =  2.0
x[3]  =  1.0
x[4]  =  4.0

Problem solved in 21899.000000 milliseconds
Problem solved in 77 iterations
Problem solved in 7 branch-and-bound nodes


### Complete programs
Here are the complete programs.

In [16]:
def main():
    data = create_data_model()
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    infinity = solver.infinity()
    x = {}
    for j in range(data['num_vars']):
        x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
    print('Number of variables =', solver.NumVariables())

    for i in range(data['num_constraints']):
        constraint_expr = []
        for j in range(data['num_vars']):
            constraint_expr.append(data['constraint_coeffs'][i][j] * x[j])
        solver.Add(sum(constraint_expr) <= data['bounds'][i])


    obj_expr = []
    for j in range(data['num_vars']):
        obj_expr.append(data['obj_coeffs'][j] * x[j])
    solver.Maximize(solver.Sum(obj_expr))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for j in range(data['num_vars']):
            print(x[j].name(), ' = ', x[j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Number of variables = 5
Objective value = 260.00000000000006
x[0]  =  9.0
x[1]  =  20.0
x[2]  =  2.0
x[3]  =  1.0
x[4]  =  4.0

Problem solved in 19.000000 milliseconds
Problem solved in 77 iterations
Problem solved in 7 branch-and-bound nodes


### Second Method

In [17]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data['bounds'] = [250, 285, 211, 315]
    data['obj_coeffs'] = [7, 8, 2, 9, 6]
    data['num_vars'] = 5
    data['num_constraints'] = 4
    return data



def main():
    data = create_data_model()
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    infinity = solver.infinity()
    x = {}
    for j in range(data['num_vars']):
        x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
    print('Number of variables =', solver.NumVariables())

    for i in range(data['num_constraints']):
        constraint_expr = [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
        solver.Add(sum(constraint_expr) <= data['bounds'][i])


    obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
    solver.Maximize(solver.Sum(obj_expr))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for j in range(data['num_vars']):
            print(x[j].name(), ' = ', x[j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Number of variables = 5
Objective value = 260.00000000000006
x[0]  =  9.0
x[1]  =  20.0
x[2]  =  2.0
x[3]  =  1.0
x[4]  =  4.0

Problem solved in 19.000000 milliseconds
Problem solved in 77 iterations
Problem solved in 7 branch-and-bound nodes


### Load an excel table in colab (This method is not for Jupyter Notebook)
Here, you can see how to import an excel table in Colab. First you need to creat the table in excel and save it as ".CSV" format. Then run the following command to load the table by selecting the table file in browser:


In [ ]:
from google.colab import files
uploaded = files.upload()

Then, you need to creat a dataframe based on the table.

In [ ]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['table.csv']))

In [ ]:
Print(data)